In [ ]:
import pandas as pd
from neo4j import GraphDatabase

import os
from dotenv import load_dotenv
from google import genai

import gradio as gr
import sys
from io import StringIO
from contextlib import redirect_stdout, redirect_stderr

from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from typing import Optional


# Load environment variables from .env file
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Neo4j connection parameters
NEO4J_URI = "YOUR_NEO4J_URL" 
NEO4J_USER = "YOUR_NEO4J_USERNAME"
NEO4J_PASSWORD = "YOUR_NEO4J_PASSWORD"

# Global driver variable
driver = None


In [26]:
data = pd.read_csv(r"Trade data\ex_im_data.csv")
export_data =pd.read_csv(r"Trade data\export_data.csv")
import_data = pd.read_csv(r"Trade data\import_data.csv")


### Creating Neo4j Graph

In [3]:
def connect_neo4j(uri, user, password):
    """Connect to Neo4j database"""
    global driver
    driver = GraphDatabase.driver(uri, auth=(user, password))
    print("Connected to Neo4j successfully!")
    return driver

def close_neo4j_connection():
    """Close Neo4j connection"""
    global driver
    if driver:
        driver.close()
        print("Neo4j connection closed!")
        driver = None


In [4]:
def check_database_has_data():
    """Check if the database already has data (countries or product groups)"""
    global driver
    if not driver:
        raise Exception("Not connected to Neo4j. Call connect_neo4j() first.")
    
    with driver.session() as session:
        result = session.run("MATCH (n) RETURN count(n) as count")
        count = result.single()["count"]
        return count > 0

def clear_database():
    """Clear all nodes and relationships from the database"""
    global driver
    if not driver:
        raise Exception("Not connected to Neo4j. Call connect_neo4j() first.")
    
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")
        print("Database cleared successfully!")


In [5]:
def create_country_node(session, country_name):
    """Create a Country node if it doesn't exist"""
    session.run(
        "MERGE (c:Country {name: $name})",
        name=country_name
    )

def create_product_group_node(session, product_group):
    """Create a ProductGroup node if it doesn't exist"""
    if pd.notna(product_group) and product_group:
        session.run(
            "MERGE (p:ProductGroup {name: $name})",
            name=str(product_group)
        )


In [6]:
def create_trade_relationship(session, reporter, partner, year, trade_data):
    """Create TRADES_WITH relationship between countries with trade data"""
    query = """
    MATCH (r:Country {name: $reporter})
    MATCH (p:Country {name: $partner})
    MERGE (r)-[t:TRADES_WITH {year: $year}]->(p)
    SET t.export_value = $export_value,
        t.import_value = $import_value,
        t.trade_balance = $trade_balance,
        t.export_products_count = $export_products_count,
        t.import_products_count = $import_products_count,
        t.export_share = $export_share,
        t.import_share = $import_share,
        t.ahs_weighted_avg = $ahs_weighted_avg,
        t.tariff_agreements = $tariff_agreements
    """
    
    session.run(
        query,
        reporter=reporter,
        partner=partner,
        year=int(year) if pd.notna(year) else None,
        export_value=float(trade_data.get('Export (US$ Thousand)', 0)) if pd.notna(trade_data.get('Export (US$ Thousand)', 0)) else 0,
        import_value=float(trade_data.get('Import (US$ Thousand)', 0)) if pd.notna(trade_data.get('Import (US$ Thousand)', 0)) else 0,
        trade_balance=float(trade_data.get('Trade Balance (US$ Thousand)', 0)) if pd.notna(trade_data.get('Trade Balance (US$ Thousand)', 0)) else 0,
        export_products_count=int(trade_data.get('No Of exported HS6 digit Products', 0)) if pd.notna(trade_data.get('No Of exported HS6 digit Products', 0)) else 0,
        import_products_count=int(trade_data.get('No Of imported HS6 digit Products', 0)) if pd.notna(trade_data.get('No Of imported HS6 digit Products', 0)) else 0,
        export_share=float(trade_data.get('Export Share in Total Products (%)', 0)) if pd.notna(trade_data.get('Export Share in Total Products (%)', 0)) else 0,
        import_share=float(trade_data.get('Import Share in Total Products (%)', 0)) if pd.notna(trade_data.get('Import Share in Total Products (%)', 0)) else 0,
        ahs_weighted_avg=float(trade_data.get('AHS Weighted Average (%)', 0)) if pd.notna(trade_data.get('AHS Weighted Average (%)', 0)) else 0,
        tariff_agreements=int(trade_data.get('No Of Tariff Agreement', 0)) if pd.notna(trade_data.get('No Of Tariff Agreement', 0)) else 0
    )


In [7]:
def create_export_relationship(session, reporter, partner, year, product_group, export_value, export_share):
    """Create EXPORTS relationship between country and product group"""
    if pd.notna(product_group) and product_group:
        query = """
        MATCH (r:Country {name: $reporter})
        MATCH (p:ProductGroup {name: $product_group})
        MERGE (r)-[e:EXPORTS {year: $year, partner: $partner}]->(p)
        SET e.export_value = $export_value,
            e.export_share = $export_share
        """
        
        session.run(
            query,
            reporter=reporter,
            partner=partner,
            product_group=str(product_group),
            year=int(year) if pd.notna(year) else None,
            export_value=float(export_value) if pd.notna(export_value) else 0,
            export_share=float(export_share) if pd.notna(export_share) else 0
        )

def create_import_relationship(session, reporter, partner, year, product_group, import_value, import_share):
    """Create IMPORTS relationship between country and product group"""
    if pd.notna(product_group) and product_group:
        query = """
        MATCH (r:Country {name: $reporter})
        MATCH (p:ProductGroup {name: $product_group})
        MERGE (r)-[i:IMPORTS {year: $year, partner: $partner}]->(p)
        SET i.import_value = $import_value,
            i.import_share = $import_share
        """
        
        session.run(
            query,
            reporter=reporter,
            partner=partner,
            product_group=str(product_group),
            year=int(year) if pd.notna(year) else None,
            import_value=float(import_value) if pd.notna(import_value) else 0,
            import_share=float(import_share) if pd.notna(import_share) else 0
        )


In [8]:
def build_graph_from_dataframes(ex_im_df, export_df, import_df):
    """Build Neo4j graph from all three dataframes"""
    global driver
    if not driver:
        raise Exception("Not connected to Neo4j. Call connect_neo4j() first.")
    
    print("Starting graph construction...")
    
    with driver.session() as session:
        # Process ex_im_data.csv - Create countries and trade relationships
        print("Processing ex_im_data...")
        for idx, row in ex_im_df.iterrows():
            reporter = row['Reporter Name']
            partner = row['Partner Name']
            
            # Create country nodes
            create_country_node(session, reporter)
            create_country_node(session, partner)
            
            # Create trade relationship
            create_trade_relationship(session, reporter, partner, row['Year'], row)
            
            if (idx + 1) % 10 == 0:
                print(f"  Processed {idx + 1} rows from ex_im_data...")
        
        print(f"Completed ex_im_data: {len(ex_im_df)} rows processed")
        
        # Process export_data.csv - Create product groups and export relationships
        print("Processing export_data...")
        for idx, row in export_df.iterrows():
            reporter = row['Reporter Name']
            partner = row['Partner Name']
            product_group = row['Product Group']
            
            # Create country nodes
            create_country_node(session, reporter)
            create_country_node(session, partner)
            
            # Create product group node
            create_product_group_node(session, product_group)
            
            # Create export relationship
            create_export_relationship(
                session, 
                reporter, 
                partner, 
                row['Year'], 
                product_group,
                row['Export (US$ Thousand)'],
                row['Export Product Share (%)']
            )
            
            if (idx + 1) % 20 == 0:
                print(f"  Processed {idx + 1} rows from export_data...")
        
        print(f"Completed export_data: {len(export_df)} rows processed")
        
        # Process import_data.csv - Create import relationships
        print("Processing import_data...")
        for idx, row in import_df.iterrows():
            reporter = row['Reporter Name']
            partner = row['Partner Name']
            product_group = row['Product Group']
            
            # Create country nodes
            create_country_node(session, reporter)
            create_country_node(session, partner)
            
            # Create product group node
            create_product_group_node(session, product_group)
            
            # Create import relationship
            create_import_relationship(
                session, 
                reporter, 
                partner, 
                row['Year'], 
                product_group,
                row['Import (US$ Thousand)'],
                row['Import Product Share (%)']
            )
            
            if (idx + 1) % 20 == 0:
                print(f"  Processed {idx + 1} rows from import_data...")
        
        print(f"Completed import_data: {len(import_df)} rows processed")
    
    print("\nGraph construction completed successfully!")


In [9]:
def execute_query(query, parameters=None):
    """Execute a Cypher query and return results"""
    global driver
    if not driver:
        raise Exception("Not connected to Neo4j. Call connect_neo4j() first.")
    
    with driver.session() as session:
        if parameters:
            result = session.run(query, parameters)
        else:
            result = session.run(query)
        
        return [record for record in result]

def query_to_dataframe(query, parameters=None):
    """Execute a Cypher query and return results as pandas DataFrame"""
    records = execute_query(query, parameters)
    if records:
        return pd.DataFrame([dict(record) for record in records])
    return pd.DataFrame()


In [10]:
# Example query functions

def get_all_product_groups():
    """Get all product groups in the graph"""
    query = "MATCH (p:ProductGroup) RETURN p.name as product_group ORDER BY p.name"
    return query_to_dataframe(query)

def get_product_trade_details(product_name: str):
    """Get trade details for a specific product: country pairs and export/import values"""
    # Get exports: exporter -> partner pairs
    export_query = """
    MATCH (exporter:Country)-[e:EXPORTS]->(p:ProductGroup {name: $product_name})
    RETURN exporter.name as exporter, e.partner as partner, e.year as year, e.export_value as value
    ORDER BY e.export_value DESC
    """
    
    # Get imports: importer -> partner pairs
    import_query = """
    MATCH (importer:Country)-[i:IMPORTS]->(p:ProductGroup {name: $product_name})
    RETURN importer.name as importer, i.partner as partner, i.year as year, i.import_value as value
    ORDER BY i.import_value DESC
    """
    
    exports_df = query_to_dataframe(export_query, parameters={'product_name': product_name})
    imports_df = query_to_dataframe(import_query, parameters={'product_name': product_name})
    
    return exports_df, imports_df

def get_trade_partners(country_name):
    """Get all trade partners for a specific country"""
    query = """
    MATCH (c:Country {name: $country_name})-[t:TRADES_WITH]->(p:Country)
    RETURN p.name as partner, 
           t.year as year,
           t.export_value as export_value,
           t.import_value as import_value,
           t.trade_balance as trade_balance
    ORDER BY t.export_value DESC
    """
    return query_to_dataframe(query, parameters={'country_name': country_name})

def get_top_export_partners(country_name, limit=10):
    """Get top export partners for a country"""
    query = """
    MATCH (c:Country {name: $country_name})-[t:TRADES_WITH]->(p:Country)
    RETURN p.name as partner,
           t.year as year,
           t.export_value as export_value,
           t.import_value as import_value,
           t.trade_balance as trade_balance
    ORDER BY t.export_value DESC
    LIMIT $limit
    """
    return query_to_dataframe(query, parameters={'country_name': country_name, 'limit': limit})

def get_top_import_partners(country_name, limit=10):
    """Get top import partners for a country"""
    query = """
    MATCH (c:Country {name: $country_name})-[t:TRADES_WITH]->(p:Country)
    RETURN p.name as partner,
           t.year as year,
           t.export_value as export_value,
           t.import_value as import_value,
           t.trade_balance as trade_balance
    ORDER BY t.import_value DESC
    LIMIT $limit
    """
    return query_to_dataframe(query, parameters={'country_name': country_name, 'limit': limit})

def get_exported_products(country_name, partner_name=None):
    """Get products exported by a country, optionally filtered by partner"""
    if partner_name:
        query = """
        MATCH (c:Country {name: $country_name})-[e:EXPORTS {partner: $partner_name}]->(p:ProductGroup)
        RETURN p.name as product_group,
               e.year as year,
               e.export_value as export_value,
               e.export_share as export_share
        ORDER BY e.export_value DESC
        """
        return query_to_dataframe(query, parameters={'country_name': country_name, 'partner_name': partner_name})
    else:
        query = """
        MATCH (c:Country {name: $country_name})-[e:EXPORTS]->(p:ProductGroup)
        RETURN p.name as product_group,
               e.partner as partner,
               e.year as year,
               e.export_value as export_value,
               e.export_share as export_share
        ORDER BY e.export_value DESC
        """
        return query_to_dataframe(query, parameters={'country_name': country_name})

def get_imported_products(country_name, partner_name=None):
    """Get products imported by a country, optionally filtered by partner"""
    if partner_name:
        query = """
        MATCH (c:Country {name: $country_name})-[i:IMPORTS {partner: $partner_name}]->(p:ProductGroup)
        RETURN p.name as product_group,
               i.year as year,
               i.import_value as import_value,
               i.import_share as import_share
        ORDER BY i.import_value DESC
        """
        return query_to_dataframe(query, parameters={'country_name': country_name, 'partner_name': partner_name})
    else:
        query = """
        MATCH (c:Country {name: $country_name})-[i:IMPORTS]->(p:ProductGroup)
        RETURN p.name as product_group,
               i.partner as partner,
               i.year as year,
               i.import_value as import_value,
               i.import_share as import_share
        ORDER BY i.import_value DESC
        """
        return query_to_dataframe(query, parameters={'country_name': country_name})

def get_trade_statistics(country_name):
    """Get comprehensive trade statistics for a country"""
    query = """
    MATCH (c:Country {name: $country_name})-[t:TRADES_WITH]->(p:Country)
    WITH c, 
         SUM(t.export_value) as total_exports,
         SUM(t.import_value) as total_imports,
         SUM(t.trade_balance) as total_balance,
         COUNT(t) as num_partners
    RETURN c.name as country,
           total_exports,
           total_imports,
           total_balance,
           num_partners
    """
    return query_to_dataframe(query, parameters={'country_name': country_name})

def find_trade_paths(country1, country2, max_depth=3):
    """Find trade paths between two countries"""
    query = """
    MATCH path = shortestPath((c1:Country {name: $country1})-[*1..$max_depth]-(c2:Country {name: $country2}))
    RETURN path
    LIMIT 10
    """
    return execute_query(query, parameters={'country1': country1, 'country2': country2, 'max_depth': max_depth})


In [ ]:
## Connecting to Neo4J
connect_neo4j(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)


Connected to Neo4j successfully!


In [ ]:
# Check if database already has data
# has_data = check_database_has_data()

# if has_data:
#     print("Database already contains data. Skipping clear and build steps.")
  
# else:
#     print("Database is empty. Need to Build graph...")

# Clear existing data (commented out by default - uncomment if you want to start fresh)
# clear_database()


Database cleared successfully!


In [28]:
build_graph_from_dataframes(data, export_data, import_data)

Starting graph construction...
Processing ex_im_data...
  Processed 10 rows from ex_im_data...
  Processed 20 rows from ex_im_data...
  Processed 30 rows from ex_im_data...
  Processed 40 rows from ex_im_data...
  Processed 50 rows from ex_im_data...
  Processed 60 rows from ex_im_data...
  Processed 70 rows from ex_im_data...
  Processed 80 rows from ex_im_data...
  Processed 90 rows from ex_im_data...
  Processed 100 rows from ex_im_data...
  Processed 110 rows from ex_im_data...
  Processed 120 rows from ex_im_data...
  Processed 130 rows from ex_im_data...
  Processed 140 rows from ex_im_data...
  Processed 150 rows from ex_im_data...
  Processed 160 rows from ex_im_data...
  Processed 170 rows from ex_im_data...
  Processed 180 rows from ex_im_data...
  Processed 190 rows from ex_im_data...
  Processed 200 rows from ex_im_data...
Completed ex_im_data: 200 rows processed
Processing export_data...
  Processed 20 rows from export_data...
  Processed 40 rows from export_data...
  Proc

### Making Agentic System to connect Graph with LLM

In [29]:
# Create CrewAI Tools using functions (no classes/OOP)
@tool("Get_trade_details_for_a_product")
def get_all_product_groups_tool(product_name: str) -> str:
    """Get trade details for a specific product including country pairs and export/import values. 
    
    Description:
        Returns all country pairs (exporter-partner and importer-partner) with their 
        export/import values for the specified product.
    
    Parameters:
        product_name (str): Name of the product group to query (e.g., "Food", "Chemicals")
    
    Returns:
        str: Formatted string showing export and import pairs with values, years, and countries
    """
    try:
        exports_df, imports_df = get_product_trade_details(product_name)
        
        result_lines = [f"Product: {product_name}\n"]
        
        # Export pairs
        if not exports_df.empty:
            result_lines.append("EXPORTS:")
            for _, row in exports_df.iterrows():
                exporter = row['exporter']
                partner = row['partner']
                year = row['year']
                value = row['value']
                result_lines.append(f"  {exporter} -> {partner} (Year: {year}): ${value:,.2f}")
        else:
            result_lines.append("EXPORTS: No export data found")
        
        result_lines.append("")
        
        # Import pairs
        if not imports_df.empty:
            result_lines.append("IMPORTS:")
            for _, row in imports_df.iterrows():
                importer = row['importer']
                partner = row['partner']
                year = row['year']
                value = row['value']
                result_lines.append(f"  {importer} -> {partner} (Year: {year}): ${value:,.2f}")
        else:
            result_lines.append("IMPORTS: No import data found")
        
        return "\n".join(result_lines)
    except Exception as e:
        return f"Error: {str(e)}"

@tool("Get_detail_about_trade_partner_for_a_country")
def get_trade_partners_tool(country_name: str) -> str:
    """Get all detail about trade partner for a specific country.
    
    Description:
        Returns all countries that trade with the given country, including export/import 
        values, trade balance, and year information.
    
    Parameters:
        country_name (str): Name of the country to query
    
    Returns:
        str: Formatted string with partner countries, years, export/import values, and trade balance
    """
    try:
        df = get_trade_partners(country_name)
        if df.empty:
            return f"No trade partners found for {country_name}."
        return df.to_string(index=False)
    except Exception as e:
        return f"Error: {str(e)}"

@tool("Get_top_export_partners_for_a_country")
def get_top_export_partners_tool(country_name: str, limit: int = 10) -> str:
    """Get top export partners for a country.
    
    Description:
        Returns top N countries that the given country exports to, sorted by export value 
        in descending order.
    
    Parameters:
        country_name (str): Name of the country to query
        limit (int): Number of top partners to return (default: 10)
    
    Returns:
        str: Formatted string with top export partners, years, export/import values, and trade balance
    """
    try:
        df = get_top_export_partners(country_name, limit)
        if df.empty:
            return f"No export partners found for {country_name}."
        return df.to_string(index=False)
    except Exception as e:
        return f"Error: {str(e)}"

@tool("Get_top_import_partners_for_a_country")
def get_top_import_partners_tool(country_name: str, limit: int = 10) -> str:
    """Get top import partners for a country.
    
    Description:
        Returns top N countries that the given country imports from, sorted by import value 
        in descending order.
    
    Parameters:
        country_name (str): Name of the country to query
        limit (int): Number of top partners to return (default: 10)
    
    Returns:
        str: Formatted string with top import partners, years, export/import values, and trade balance
    """
    try:
        df = get_top_import_partners(country_name, limit)
        if df.empty:
            return f"No import partners found for {country_name}."
        return df.to_string(index=False)
    except Exception as e:
        return f"Error: {str(e)}"

@tool("Get_products_exported_by_country_or_between_two_countries")
def get_exported_products_tool(country_name: str, partner_name: Optional[str] = None) -> str:
    """Get products exported by a country or between two countries if partner name is provided.
    
    Description:
        Returns all products exported by a country. Optionally filter results to show only 
        exports to a specific partner country.
    
    Parameters:
        country_name (str): Name of the exporting country
        partner_name (Optional[str]): Optional partner country name to filter exports (default: None)
    
    Returns:
        str: Formatted string with product groups, years, export values, and export shares
    """
    try:
        df = get_exported_products(country_name, partner_name)
        if df.empty:
            return f"No exported products found for {country_name}."
        return df.to_string(index=False)
    except Exception as e:
        return f"Error: {str(e)}"

@tool("Get_products_imported_by_country_or_between_two_countries")
def get_imported_products_tool(country_name: str, partner_name: Optional[str] = None) -> str:
    """Get products imported by a country or between two countries if partner name is provided
    
    Description:
        Returns all products imported by a country. Optionally filter results to show only 
        imports from a specific partner country.
    
    Parameters:
        country_name (str): Name of the importing country
        partner_name (Optional[str]): Optional partner country name to filter imports (default: None)
    
    Returns:
        str: Formatted string with product groups, years, import values, and import shares
    """
    try:
        df = get_imported_products(country_name, partner_name)
        if df.empty:
            return f"No imported products found for {country_name}."
        return df.to_string(index=False)
    except Exception as e:
        return f"Error: {str(e)}"

@tool("Get_trade_statistics_for_a_country")
def get_trade_statistics_tool(country_name: str) -> str:
    """Get comprehensive trade statistics for a country.
    
    Description:
        Returns aggregated trade statistics including total exports, total imports, 
        trade balance, and number of trading partners.
    
    Parameters:
        country_name (str): Name of the country to query
    
    Returns:
        str: Formatted string with total exports, total imports, trade balance, and partner count
    """
    try:
        df = get_trade_statistics(country_name)
        if df.empty:
            return f"No trade statistics found for {country_name}."
        return df.to_string(index=False)
    except Exception as e:
        return f"Error: {str(e)}"

@tool("Find_trade_paths_between_two_countries")
def find_trade_paths_tool(country1: str, country2: str, max_depth: int = 4) -> str:
    """Find trade paths between two countries.
    
    Description:
        Finds shortest trade connection paths between two countries through intermediate 
        trading partners in the graph database.
    
    Parameters:
        country1 (str): Name of the first country
        country2 (str): Name of the second country
        max_depth (int): Maximum depth for path search (default: 4)
    
    Returns:
        str: Formatted string showing trade paths connecting the two countries
    """
    try:
        results = find_trade_paths(country1, country2, max_depth)
        if not results:
            return f"No trade paths found between {country1} and {country2}."
        # Format the path results
        formatted_results = []
        for record in results:
            path = record.get('path', {})
            formatted_results.append(str(path))
        return "\n".join(formatted_results)
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:

neo4j_tools = [
  
    get_all_product_groups_tool,
    get_trade_partners_tool,
    get_top_export_partners_tool,
    get_top_import_partners_tool,
    get_exported_products_tool,
    get_imported_products_tool,
    get_trade_statistics_tool,
    find_trade_paths_tool
]


In [ ]:

llm = LLM(
        model="gemini/gemini-2.0-flash",
        provider="google_gemini",                   
        api_key= gemini_api_key,
        temperature=0.5
    )


In [ ]:
trade_data_analyst = Agent(
    role="Neo4j Trade Data Analyst",
    goal=(
        """Accurately retrieve and summarize factual trade data from the Neo4j graph database 
        to support user queries about exports, imports, products, and bilateral trade relationships."""
    ),
    backstory=(
        """You are a highly skilled data analyst specializing in using tools to extract international trade data.

All factual information must come exclusively from the system’s tools, which execute Neo4j queries and return results as plain text. You must not invent or approximate numbers — always call the appropriate tool to obtain real data from the database.

When the user mentions countries, you must convert them into the exact query format expected by the tools:

“India”, “india” → India

“US” → United States

“china”, “China” → China

“UAE” → United Arab Emirates

Your role is to answer using verified database results, ensuring accuracy and avoiding any hallucinated data."""
    ),
    tools=neo4j_tools,      
    max_iter=4,
    llm=llm,                 
    verbose=True,
    allow_delegation=False
)


In [33]:
# Create a task for answering user queries
task_fetch_trade_data = Task(
    description=(
        """Given the user's natural language query, use the Neo4j tools to fetch the 
        explicit data needed to answer the question. 
        ----------------------------------------------------------------------
        The user's <QUERY> is: \n{query}"""
    ),
    agent=trade_data_analyst,
    expected_output="A comprehensive answer to the user's query with relevant data from the Neo4j database, formatted clearly and with appropriate context."
    )


In [34]:
llm_trade_advisor = Agent(
    role="LLM Trade Advisor",
    goal=(
        "Provide an interpretive, explanatory, or conceptual answer to user questions using the real world factual data about trades relation between countries."
    ),
    backstory=('''You are an expert economist and trade-policy specialist with strong 
    knowledge of international trade dynamics. You understand concepts such as trade
     balances, comparative advantage, tariff structures, HS classifications, 
     supply-chain impacts, sanctions, and trade agreements. You can also analyze 
     geopolitical and macroeconomic factors that influence trade relationships. 
     Your role is to explain trade-related questions clearly, assess causes and 
     consequences, and provide high-level, policy-relevant insights.'''
    ),
    tools=[],                # IMPORTANT: no tools
    max_iter=2,
    llm=llm,
    verbose=True,
    allow_delegation=False
)


In [35]:
task_synthesize_response = Task(
    description=(
        """Take the user's original natural-language question plus the structured factual payload returned by 
        the Trade Data Analyst agent and produce a clear, helpful, and non-hallucinated final answer for a given user <QUERY>:
        ---------------------------------------------------------------------------------------------------------------------
         <QUERY> is: \n{query}"""
       
    ),
    agent=llm_trade_advisor,
    expected_output=(
    '''Provide a clear, plain-language response that:

- accurately summarizes the facts or data provided by the user

- interprets these facts using sound economic and trade-policy reasoning

- highlights the most important insights or implications in a concise summary'''
))



In [36]:
crew = Crew(
        agents=[trade_data_analyst,llm_trade_advisor],
        tasks=[task_fetch_trade_data,task_synthesize_response],
        process= Process.sequential,
        verbose=False,
        cache=False
    )

In [41]:
# result = crew.kickoff(inputs={'query':"How does india will be impacted if US imposed tariffs on Chemicals?"})
# print(result)

### Gradio

In [ ]:

def run_query(query):
    try:
        result = crew.kickoff(inputs={'query': query})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"


# Create Gradio interface
with gr.Blocks(title="GeoTrade Intelligence Engine", theme=gr.themes.Soft()) as demo:
    gr.Markdown("<h1 style='text-align: center; font-size: 3em;'>🌍 GeoTrade Intelligence Engine</h1>")
    gr.Markdown("<p style='text-align: center; font-size: 1.2em; color: #666;'>Ask anything about international trade or macroeconomics — every answer grounded in real graph data.</p>")
    
    with gr.Row():
        with gr.Column(scale=0.1):
            gr.Markdown("### 📊 Technical Overview\n**Data Source:** World Trade Center trade data\n\n**Database:** Neo4j Graph Database\n\n**Framework:** CrewAI Agentic System")
        
        with gr.Column(scale=5):
            query_input = gr.Textbox(
                label="Enter your query:",
                placeholder="e.g., What are the top export partners for India?",
                lines=5
            )
            
            run_btn = gr.Button("🚀 Run", variant="primary")
            result_output = gr.Textbox(label="📋 Results", lines=15, interactive=False)
    run_btn.click(fn=run_query, inputs=query_input, outputs=result_output)

# Launch interface
demo.launch(share=False, server_name="127.0.0.1")


c:\Users\GenAI\Downloads\Cursor\myenv\Lib\site-packages\gradio\layouts\column.py:59: UserWarning: 'scale' value should be an integer. Using 0.1 will cause issues.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Neo4j Trade Data Analyst                                                                                │
│                                                                                                                 │
│  Task: Given the user's natural language query, use the Neo4j tools to fetch the                                │
│          explicit data needed to answer the question.                                                           │
│          ----------------------------------------------------------------------                                 │
│          The user's <QUERY> is:                                                                                 │
│  How India will be impacted if US raises tariffs on Chemical?                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Neo4j Trade Data Analyst                                                                                │
│                                                                                                                 │
│  Thought: The user is asking about the impact on India if the US raises tariffs on Chemicals. This requires     │
│  understanding the trade relationship between India and the US for Chemicals. I need to find out how much       │
│  India exports/imports Chemicals to/from the US. I should start by getting the trade details for "Chemicals"    │
│  between India and the US.                                                                                      │
│                                                                                                                 │
│  Using Tool: Get_trade_details_for_a_product                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "product_name": "Chemicals"                                                                                  │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                      product_group  year  export_value  export_share                                            │
│  Machinery and Transport Equipment  2022    15579029.9         19.42                                            │
│                           Chemical  2022    13625232.0         16.98                                            │
│                           Textiles  2022    11092819.9         13.83                                            │
│                      Food Products  2022     1510622.2          1.88                                            │
│                    Ores and Metals  2022     1080557.0          1.35                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Neo4j Trade Data Analyst                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  If the US raises tariffs on Chemicals, India would be impacted because Chemicals exports from India to the     │
│  United States were valued at $13,625,232 in 2022, representing 16.98% of India's total exports to the United   │
│  States. Increased tariffs could make Indian chemicals more expensive in the US market, potentially reducing    │
│  the volume of exports and affecting Indian chemical manufacturers.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: LLM Trade Advisor                                                                                       │
│                                                                                                                 │
│  Task: Take the user's original natural-language question plus the structured factual payload returned by       │
│          the Trade Data Analyst agent and produce a clear, helpful, and non-hallucinated final answer for a     │
│  given user <QUERY>:                                                                                            │
│          -----------------------------------------------------------------------------------------------------  │
│  ----------------                                                                                               │
│           <QUERY> is:                                                                                           │
│  How India will be impacted if US raises tariffs on Chemical?                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Trace batch initialization returned status 401. Continuing without tracing.


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: LLM Trade Advisor                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  If the US raises tariffs on chemicals, India would be impacted because chemical exports from India to the      │
│  United States were valued at $13,625,232 in 2022, representing 16.98% of India's total exports to the United   │
│  States. Increased tariffs could make Indian chemicals more expensive in the US market, potentially reducing    │
│  the volume of exports and affecting Indian chemical manufacturers.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯